In [64]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import scale, minmax_scale
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, minmax_scale
from torch.utils.data import TensorDataset, DataLoader,Dataset

import torch.optim as optim

import torchvision
from torchvision import datasets, transforms
from torch.utils import data

# Data loading - MNSIT

In [2]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
# Sélection des targets avec pandas
A = pd.DataFrame(X)
A['y'] = y
B = A.loc[A.index[A['y']=='4'].union(A.index[A['y']=='7'])]
# Passage du dataframe pandas vers les tensors de torch 
# + normalisation des images pour que les valeurs des pixels soient entre 0 et 1
# + reshape des données en un tenseur en 4 dimensions : batch, channel, height, width
X_4_7 = torch.tensor(minmax_scale(B.drop("y", axis=1)).astype(np.float32)).reshape(B.shape[0],1,28,28)
y_4_7 = torch.tensor(B.y.replace(['4','7'],[0,1]).values.astype(np.float32)).unsqueeze(-1)

mnist_4_7 = TensorDataset(X_4_7, y_4_7) # on lie les entrées aux targets attentues
trainloader = DataLoader(mnist_4_7) # pour faciliter l'échantillonage en mini-batch

# Architecture 1

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28, 28)
        self.fc2 = nn.Linear(28, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=28, out_features=28, bias=True)
  (fc2): Linear(in_features=28, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)


In [41]:
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
                
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        print('inputs: ', inputs.shape)
        print('labels: ', labels.shape, labels)
        print('outputs: ', outputs.shape, outputs)    
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

inputs:  torch.Size([1, 1, 28, 28])
labels:  torch.Size([1, 1]) tensor([[0.]])
outputs:  torch.Size([1, 1, 28, 1]) tensor([[[[0.4856],
          [0.4856],
          [0.4856],
          [0.4856],
          [0.4856],
          [0.4881],
          [0.4851],
          [0.4842],
          [0.4731],
          [0.4776],
          [0.4752],
          [0.4748],
          [0.4712],
          [0.4741],
          [0.4822],
          [0.5018],
          [0.4918],
          [0.4879],
          [0.4854],
          [0.4854],
          [0.4866],
          [0.4867],
          [0.4883],
          [0.4882],
          [0.4862],
          [0.4856],
          [0.4856],
          [0.4856]]]], grad_fn=<SigmoidBackward>)


ValueError: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 1, 28, 1])) is deprecated. Please ensure they have the same size.

# Data loading - FashionMNSIT

In [77]:
def load_data_fashion_mnist(batch_size, resize=None):  #@save
    """Download the Fashion-MNIST dataset and then load it into memory."""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=2),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=2))

In [78]:
train_iter, test_iter = load_data_fashion_mnist(32, resize=64)
for X, y in train_iter:
    print(X.shape, X.dtype, y.shape, y.dtype)
    break

AttributeError: 'list' object has no attribute 'DataLoader'

# Architeture 2

In [72]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net22(nn.Module):

    def __init__(self):
        super(Net22, self).__init__()
        self.fc1 = nn.Linear(28, 28)
        self.fc2 = nn.Linear(28, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x


net22 = Net22()
print(net22)

Net22(
  (fc1): Linear(in_features=28, out_features=28, bias=True)
  (fc2): Linear(in_features=28, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)


In [75]:
error = nn.BCELoss()
optimizer = torch.optim.SGD(net2.parameters(), lr=0.001)

train_iter, test_iter = load_data_fashion_mnist(32, resize=64)

for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

AttributeError: 'list' object has no attribute 'DataLoader'

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net2(nn.Module):

    def __init__(self):
        super(Net2, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=4, padding=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=32, kernel_size=4),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        
        self.fc1 = nn.Linear(in_features=32*5*5, out_features=150)
        self.fc2 = nn.Linear(in_features=150, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=1)
        

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = torch.sigmoid(self.fc3(out))
        
        return out

net2 = Net2()
print(net2)

Net2(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Flatten(start_dim=1, end_dim=-1)
  )
  (fc1): Linear(in_features=800, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)


In [68]:
error = nn.BCELoss()
optimizer = torch.optim.SGD(net2.parameters(), lr=0.001)

In [73]:
train_iter, test_iter = load_data_fashion_mnist(32, resize=64)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for X, y in train_iter:
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net2(X)
        
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

AttributeError: 'list' object has no attribute 'DataLoader'

# V2

# Data Load 2

In [62]:
from sklearn.datasets import fetch_openml
X, y  = fetch_openml(data_id=40996, return_X_y=True, as_frame=False) 

A = pd.DataFrame(X)
A['y'] = y

B = A.loc[A.index[A['y']=='9'].union(A.index[A['y']=='5'])]
X_5_9 = torch.tensor(minmax_scale(B.drop("y", axis=1)).astype(np.float32)).reshape(B.shape[0],1,28,28)
y_5_9 = torch.tensor(B.y.replace(['5','9'],[0,1]).values.astype(np.float32)).unsqueeze(-1)
y_5_9

tensor([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [0.]])

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X_5_9, y_5_9, test_size=0.20, random_state=42)

train_set = TensorDataset(X_train, y_train)
test_set = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_set, shuffle=True)
test_loader = DataLoader(test_set, shuffle=True)

# Net2

In [70]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 0.444
[1,  4000] loss: 0.184
[1,  6000] loss: 0.151
[1,  8000] loss: 0.108
[1, 10000] loss: 0.136
[2,  2000] loss: 0.113
[2,  4000] loss: 0.095
[2,  6000] loss: 0.099
[2,  8000] loss: 0.103
[2, 10000] loss: 0.083


In [81]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(test_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 0.077
[2,  2000] loss: 0.077


# Net 22

In [82]:
error = nn.BCELoss()
optimizer = torch.optim.SGD(net2.parameters(), lr=0.001)

train_loader = DataLoader(train_set, shuffle=True)
test_loader = DataLoader(test_set, shuffle=True)


for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net22(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

ValueError: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 1, 28, 1])) is deprecated. Please ensure they have the same size.

# Remarques

- J'ai encore qlqs problèmes avec le chargement des données/tensor.
- On remarque bien que l'erreur diminue au cours du train. (Net2)